In [59]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import random
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW 
import json
import torch
import numpy as np
import math
test = pd.read_csv('./sample_submission.csv')
model_path = './bert/'

In [2]:
# nltk.download()
words = stopwords.words('english')
print(words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
doc = pd.read_csv('./documents.csv')
doc_list = doc['doc_id'].tolist()
doc_text = doc['doc_text'].tolist()
doc_dict = {doc_list[i]: i for i in range(len(doc_list))}
# print(len(doc_text))

In [4]:
train_q = pd.read_csv('./train_queries.csv')
train_query_data = [t.lower() for t in train_q['query_text']]
train_top_1000 = [t.split() for t in train_q['bm25_top1000']]
train_pos_list = [t.split() for t in train_q['pos_doc_ids']]
train_neg_list = [t[:] for t in train_top_1000] 
train_pos_list = np.array(train_pos_list)
train_top_1000 = np.array(train_top_1000)
train_neg_list = np.array(train_neg_list) 
# print(train_q)
# print(train_top_1000[0])

/home/nlp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [5]:
## 裡面現在存的是pos doc idx
for pos_docs in range(train_pos_list.shape[0]):
    for idx in range(len(train_pos_list[pos_docs])):
        train_pos_list[pos_docs][idx] = doc_dict[train_pos_list[pos_docs][idx]]
        
for pos_docs in range(train_top_1000.shape[0]):
    for idx in range(len(train_top_1000[pos_docs])):
        train_top_1000[pos_docs][idx] = doc_dict[train_top_1000[pos_docs][idx]]
        
for pos_docs in range(train_neg_list.shape[0]):
    for idx in range(len(train_neg_list[pos_docs])):
        train_neg_list[pos_docs][idx] = doc_dict[train_neg_list[pos_docs][idx]]


In [6]:
def clean_string(content):
    result = ""
    content = content.replace('\n','.').replace('\t',',')
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content=  content.lower()
    stopword = {}
    for i in stopwords.words('english'):
        stopword[i] = 0
    for word in content.split():
        if word not in stopword.keys():
            result += word
            result += " "
    return result

In [7]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return input_id, tokentype, attentionmask, y
    def __len__(self):
        return len(self.input_ids)

In [60]:
train_question = []
train_choice = []
train_y = []
N = 3 ## 錯的文章數量
for i in range(len(train_query_data)):
    if len(train_pos_list[i]) <50:
        N = 10
    else:
        N = 3
    for j in range(len(train_pos_list[i])): ##幾篇POS
        if type(doc_text[train_pos_list[i][j]]) != float:
            train_y.append(1)
            train_question.append(train_query_data[i])
            train_choice.append(clean_string(doc_text[train_pos_list[i][j]])) ##對的
#             upper_bound = (N+1)*len(train_pos_list[i])
#             if upper_bound>=999:
#                 upper_bound = 999
            l = random.sample(range(0,999), N)
            count = -1
            while(count < N): 
                if count != -1:
                    l = random.sample(range(0,999), N)
                count = 0 
                for check in l:
                    if int(train_neg_list[i][check]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][check])]) != float: ##不是POS
                        count += 1    
            for idx in l:
                train_y.append(0)
                train_question.append(train_query_data[i])
                train_choice.append(clean_string(doc_text[int(train_neg_list[i][idx])]))
train_y = np.array(train_y)

In [8]:
# train_question = []
# train_choice = []
# train_y = []
# R = 1 ##偽相關的數量
# N = 2 ## 錯的文章數量
# for i in range(len(train_query_data)):
# #     print(i)
#     for j in range(len(train_pos_list[i])): ##幾篇POS
#         if type(doc_text[train_pos_list[i][j]]) != float:
#             train_y.append(2) ##代表相關
#             train_question.append(train_query_data[i])
#             train_choice.append(clean_string(doc_text[train_pos_list[i][j]])) ##對的
# #             upper_bound = (N+1)*len(train_pos_list[i])
# #             if upper_bound>=999:
# #                 upper_bound = 999
#             n = random.sample(range(0,99),R)
#             flag = True
#             while(flag):
#                 if int(train_neg_list[i][n[0]]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][n[0]])]) != float: ##不是POS
#                     flag = False  
#                 else:
#                     n = random.sample(range(0,99),R)
#             train_y.append(1)
#             train_question.append(train_query_data[i])
#             train_choice.append(clean_string(doc_text[int(train_neg_list[i][n[0]])]))
#             l = random.sample(range(100,999), N)
#             count = -1
#             while(count < N): 
#                 if count != -1:
#                     l = random.sample(range(100,999), N)
#                 count = 0 
#                 for check in l:
#                     if int(train_neg_list[i][check]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][check])]) != float: ##不是POS
#                         count += 1    
#             for idx in l:
#                 train_y.append(0)
#                 train_question.append(train_query_data[i])
#                 train_choice.append(clean_string(doc_text[int(train_neg_list[i][idx])]))
# train_y = np.array(train_y)

In [61]:
print(train_question[0])
print(len(train_question))
print(len(train_choice))
print(train_y[:100])
print(train_y.shape)

poliomyelitis and post-polio
46743
46743
[1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
(46743,)


In [ ]:
# ## 備份
# train_question = []
# train_choice = []
# train_y = []
# N = 3 ## 錯的文章數量
# for i in range(len(train_query_data)):
#     for j in range(len(train_pos_list[i])):
#         if type(doc_text[train_pos_list[i][j]]) != float:
#             query = ["","","",""]
#             choice = ["","","",""]
#             ans_pos = random.randint(0,3) ##隨機給正確答案的位置
#             train_y.append(ans_pos)
#             query[ans_pos] = train_query_data[i]
#             choice[ans_pos] = clean_string(doc_text[train_pos_list[i][j]])
#             l = random.sample(range(0,100000), N)
#             count = -1
#             while(count<3): ##有抽到前1000的
#                 if count != -1:
#                     l = random.sample(range(0,100000), N)
#                 count = 0 
#                 for check in l:
#                     if check not in train_top_1000[i] or type(doc_text[check]) != float: ##不是top 1000
#                         count += 1        
#             for idx in range(N+1):
#                 if idx != ans_pos:
#                     query[idx] = train_query_data[i]
#                     choice[idx] = clean_string(doc_text[idx])
#             query = json.dumps(query)
#             choice = json.dumps(choice)
#             train_question.append(query)
#             train_choice.append(choice)
# #             train_question.append(json.dumps(query))
# #             train_choice.append(json.dumps(choice))
# # train_question = np.array(train_question)
# # train_choice = np.array(train_choice)
# print(train_question[0])
# train_y = np.array(train_y)
# # print(train_question.shape)
# # print(train_choice.shape)
# print(train_y.shape)http://localhost:8787/notebooks/NewsClassify/Mouth/HW6.ipynb#

In [72]:
## model 各參數
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_labels = 2
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
# model.load_state_dict(torch.load('./HW6_model/HW6_baseline_0.pkl'))
model = model.to(device)
model.train()
EVAL = 9000
EPOCHS = 10
optimizer = AdamW(model.parameters(),lr = 1e-5)

Some weights of the model checkpoint at ./bert/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert/ a

In [28]:
# ## model 各參數
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# num_labels = 2
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
# model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased',num_labels = num_labels)
# # model.load_state_dict(torch.load('./HW6_model/HW6_baseline_0.pkl'))
# model = model.to(device)
# model.train()
# EVAL = 7500
# EPOCHS = 20
# optimizer = AdamW(model.parameters(),lr = 1e-5)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [65]:
test_q_set = train_question[:EVAL]
test_c_set = train_choice[:EVAL]
train_q_set = train_question[:]
train_c_set = train_choice[:]

test_y_set = train_y[:EVAL]
train_y_set = train_y[:]
print(len(test_q_set))
print(len(test_c_set))
print(len(test_y_set))

print(train_q_set[2])
print(train_c_set[0])
print(len(train_y_set))

9000
9000
9000
poliomyelitis and post-polio
language: <f p=105> chinese <f> article type:cso <f p=106> [article zhao zhuliang 6392 4554 5328) central <f> china teachers college edited xu honghai 6079 1347 3189): state chinas physically challenged persons course modernization] [excerpt] [passage omitted] recent years along development chinas physically challenged persons cause condition large special group improved certain extent current state physically challenged areas economics education jobs recovery generally remains far behind average level society whole 1 threat-to-survival crisis statistics show 49 percent chinas population physically challenged many 20 million struggling survive subsistence line 70 percent physically challenged existing support family relatives state collective relief study reported 1993 2 issue zhongguo canji ren [chinas disabled persons] says that: many physically challenged remain great hardship basically enough food clothing instance shaoyang city hunan pro

In [231]:
print(test_q_set[7499])
for i in range(120):
    if train_query_data[i] == test_q_set[7499]:
        print(i)

health and computer terminals
27


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_test_acc(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           token_type_ids = segment_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def get_test_score(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    result_batch = [] ## all label 1's score
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           token_type_ids = segment_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            score = outputs[1]
            for idx in range(score.shape[0]):
                result_batch.append(score[idx][1])
#             result_score = 0
#             for num in score:
#                 for result in num:
#                     result_score += result
#             print(result_score)
    return result_batch

In [13]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# def get_test_score(model, testloader):
#     model.eval()  ##test mode
#     total = 0 ##total_num
#     result_batch = [] ## all label 1's score
#     correct = 0 ##correct_num
#     with torch.no_grad():   ##eval不計算gradient 
#         for data in testloader:
#             tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
#             outputs = model(input_ids = tokens_tensors,
#                            token_type_ids = segment_tensors,
#                            attention_mask = masks_tensors,
#                            labels = labels)
#             score = outputs[1]
#             for idx in range(score.shape[0]):
#                 result_batch.append(score[idx][2]-score[idx][1]) ##相關扣掉偽相關
#     return result_batch

In [35]:
# def MAP():
#     top_1000 = [t.split() for t in train_q['bm25_top1000']]
#     pos_ans = [t.split() for t in train_q['pos_doc_ids']]
#     num = 0
#     a = 0 
#     num_query = 120 ## 25 拿來test 
#     all_score = 0.0
#     for i in range(int(num_query)):
#         score = 0.0
#         find_list = []## all list
#         pred_list = [] ## all pred score
#         pos_list = [] ##all score changed pos 
#         relevant_list = {} ## all relevant
# #         all_find = input() ##所有query結果
#         all_find_split = top_1000[i] ## list all find
#         for j in top_1000[i]:
#             find_list.append(j) ## 建list
#             pred_list.append(0)
# #         relevant = input()
#         all_relevant_split = pos_ans[i] ## list all find
#         print(len(all_relevant_split))
#         num+=len(all_relevant_split)
#         if len(all_relevant_split)>80:
#             a += 1
#         for j in pos_ans[i]:
#             relevant_list[j] = 0 ## 建dict 比較好查找
#         now_pos = 0
#         correct = 0 ##對幾個
#         count = 0
#         for j in range(len(find_list)):
#             count += 1
#             if(find_list[j] in relevant_list):
#                 correct += 1
#                 for k in range(count-now_pos):
#                     pred_list[now_pos+k] = correct/count
#                 now_pos = j+1
#                 pos_list.append(j)
#         for j in range(len(pos_list)):
#             score += pred_list[pos_list[j]]
#         if len(pos_list)!=0:
#             score /= len(all_relevant_split) 
#         all_score += score
#     all_score/=int(num_query)
#     print(num)
#     print(a)
#     return (round(all_score,4))

In [17]:
# test_a = [t.split() for t in train_q['bm25_top1000']]
# MAP() ##要傳25x1000

In [66]:
## train
train_input_dict = tokenizer(train_q_set, train_c_set, 
                             padding=True, 
                             truncation=True, 
                             return_tensors="pt",
                             return_token_type_ids = True,
                             max_length = 512)
TRAIN_BATCH_SIZE = 4
trainset = TrainDataset(train_input_dict, train_y_set) ##trainset參數如init
trainloader = DataLoader(trainset, batch_size = TRAIN_BATCH_SIZE, shuffle = True)  




In [67]:
## test 
test_input_dict = tokenizer(test_q_set, test_c_set, 
                             padding=True, 
                             truncation=True, 
                             return_tensors="pt",
                             return_token_type_ids = True,
                             max_length = 512)
TEST_BATCH_SIZE = 64
testset = TrainDataset(test_input_dict, test_y_set) ##trainset參數如init
testloader = DataLoader(testset, batch_size = TEST_BATCH_SIZE, shuffle = True)  

In [70]:
def new_trainloader():
    train_question = []
    train_choice = []
    train_y = []
    N = 3 ## 錯的文章數量
    EVAL = 9000
    for i in range(len(train_query_data)):
        if len(train_pos_list[i]) <50:
            N = 10
        else:
            N = 3
        for j in range(len(train_pos_list[i])): ##幾篇POS
            if type(doc_text[train_pos_list[i][j]]) != float:
                train_y.append(1)
                train_question.append(train_query_data[i])
                train_choice.append(clean_string(doc_text[train_pos_list[i][j]])) ##對的
    #             upper_bound = (N+1)*len(train_pos_list[i])
    #             if upper_bound>=999:
    #                 upper_bound = 999
                l = random.sample(range(0,999), N)
                count = -1
                while(count < N): 
                    if count != -1:
                        l = random.sample(range(0,999), N)
                    count = 0 
                    for check in l:
                        if int(train_neg_list[i][check]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][check])]) != float: ##不是POS
                            count += 1    
                for idx in l:
                    train_y.append(0)
                    train_question.append(train_query_data[i])
                    train_choice.append(clean_string(doc_text[int(train_neg_list[i][idx])]))
    train_y = np.array(train_y)
     ###切割驗證 訓練
    test_q_set = train_question[:EVAL]
    test_c_set = train_choice[:EVAL]
    train_q_set = train_question[:]
    train_c_set = train_choice[:]

    test_y_set = train_y[:EVAL]
    train_y_set = train_y[:]
    ### tokenizer
    input_dict = tokenizer(train_q_set, train_c_set, 
                             padding=True, 
                             truncation=True, 
                             return_tensors="pt",
                             return_token_type_ids = True,
                             max_length = 512)
    TRAIN_BATCH_SIZE = 4
    data = TrainDataset(input_dict, train_y_set) ##trainset參數如init
    loader = DataLoader(data, batch_size = TRAIN_BATCH_SIZE, shuffle = True)  
    return loader

In [16]:
# def new_trainloader():
#     train_question = []
#     train_choice = []
#     train_y = []
#     R = 1 ##偽相關的數量
#     EVAL = 7500
#     N = 2 ## 錯的文章數量
#     for i in range(len(train_query_data)):
#         for j in range(len(train_pos_list[i])): ##幾篇POS
#             if type(doc_text[train_pos_list[i][j]]) != float:
#                 train_y.append(2) ##代表相關
#                 train_question.append(train_query_data[i])
#                 train_choice.append(clean_string(doc_text[train_pos_list[i][j]])) ##對的
#     #             upper_bound = (N+1)*len(train_pos_list[i])
#     #             if upper_bound>=999:
#     #                 upper_bound = 999
#                 n = random.sample(range(0,99),R)
#                 flag = True
#                 while(flag):
#                     if int(train_neg_list[i][n[0]]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][n[0]])]) != float: ##不是POS
#                         flag = False  
#                     else:
#                         n = random.sample(range(0,99),R)
#                 train_y.append(1)
#                 train_question.append(train_query_data[i])
#                 train_choice.append(clean_string(doc_text[int(train_neg_list[i][n[0]])]))
#                 l = random.sample(range(100,999), N)
#                 count = -1
#                 while(count < N): 
#                     if count != -1:
#                         l = random.sample(range(100,999), N)
#                     count = 0 
#                     for check in l:
#                         if int(train_neg_list[i][check]) not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][check])]) != float: ##不是POS
#                             count += 1    
#                 for idx in l:
#                     train_y.append(0)
#                     train_question.append(train_query_data[i])
#                     train_choice.append(clean_string(doc_text[int(train_neg_list[i][idx])]))
#     train_y = np.array(train_y)
#      ###切割驗證 訓練
#     test_q_set = train_question[:EVAL]
#     test_c_set = train_choice[:EVAL]
#     train_q_set = train_question[EVAL:]
#     train_c_set = train_choice[EVAL:]

#     test_y_set = train_y[:EVAL]
#     train_y_set = train_y[EVAL:]
#     ### tokenizer
#     input_dict = tokenizer(train_q_set, train_c_set, 
#                              padding=True, 
#                              truncation=True, 
#                              return_tensors="pt",
#                              return_token_type_ids = True,
#                              max_length = 512)
#     TRAIN_BATCH_SIZE = 4
#     data = TrainDataset(input_dict, train_y_set) ##trainset參數如init
#     loader = DataLoader(data, batch_size = TRAIN_BATCH_SIZE, shuffle = True)  
#     return loader

In [44]:
# num_labels = 3
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
# model = model.to(device)
# # model.load_state_dict(torch.load('./HW6_model/HW6_1e5_new_train_data_0.pkl'))
# EPOCHS = 5 #ten model 兩個一組
# optimizer = AdamW(model.parameters(),lr = 1e-5)
# # model.eval()
# # test_acc = get_test_acc(model,testloader)
# # print(test_acc)

Some weights of the model checkpoint at ./bert/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert/ a

In [73]:
# e = 0.1 ##label smooth的參數
EPOCHS = 10
# smooth_loss_fnc = F.CrossEntropyLoss()
type_weight = torch.FloatTensor([1.2e+00,6.0e+00]).to(device)
type_loss_func = F.CrossEntropyLoss(weight=type_weight)
origin_lr = 1e-5
model.train()
for epoch in range(EPOCHS):
    i = 0
    running_loss = 0.0
    correct = 0
    total = 0
    if epoch != 0:
        if (epoch % 2 == 0):
            num_labels = 2
            model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels) ##重新init model
            model = model.to(device)
            model.train()
            optimizer = AdamW(model.parameters(),lr = origin_lr) 
        trainloader = new_trainloader() ##重新選random
        optimizer = AdamW(model.parameters(),lr = origin_lr *(0.7)**(epoch%3)) ##隨epoch下降lr
    for (i,data) in enumerate(trainloader):
        
        tokens_tensors ,  segments_tensors , masks_tensors , labels  = [t.to(device) for t in data]
        bert_outputs = model(input_ids=tokens_tensors, 
                             token_type_ids=segments_tensors, 
                             attention_mask=masks_tensors,
                             labels = labels)

        optimizer.zero_grad()
        logits = bert_outputs[1]
        pred = torch.argmax(logits,dim = -1)

        loss = type_loss_func(logits,labels)
#         loss = bert_outputs[0]
        loss.backward()
        optimizer.step()
        total += pred.size()[0]
        correct += (pred == labels).sum().item()
        running_loss += loss.item()
        
        print(f'\rEpoch [{epoch+1}/{EPOCHS}] {i}/{len(trainloader)} Loss: {running_loss:.4f} Acc : {(correct/total):.4f}', end='')
    test_acc = get_test_acc(model,testloader)
    ##all save
    torch.save(model.state_dict(),'./HW6_model/HW6_bert_new_balance_' + str(epoch) + '.pkl')
    print('   test acc:' , test_acc)
    with open('./hw6_test_score_1000'+str(epoch)+'.txt','w') as f:
        f.write(str(test_acc))
        f.write('\n')
        f.close()

Epoch [1/10] 9422/9423 Loss: 3145.1042 Acc : 0.8556   test acc: 0.8891111111111111
Epoch [3/10] 11685/11686 Loss: 3685.1519 Acc : 0.8800   test acc: 0.935
Epoch [4/10] 11685/11686 Loss: 2431.7421 Acc : 0.9236   test acc: 0.9528888888888889


Some weights of the model checkpoint at ./bert/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert/ a

Epoch [5/10] 10811/11686 Loss: 3289.8735 Acc : 0.8795

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [9/10] 11685/11686 Loss: 3445.2270 Acc : 0.8824   test acc: 0.9208888888888889
Epoch [10/10] 2363/11686 Loss: 477.8781 Acc : 0.9240

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
num_labels = 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.load_state_dict(torch.load('./HW6_model/HW6_bert_balance_12.pkl')) ##先拿中間ㄉ
model.eval()
# test_acc = get_test_acc(model,testloader)
# print(test_acc)

Some weights of the model checkpoint at ./bert/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert/ a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [94]:
# num_labels = 2
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
# model = model.to(device)
# model.load_state_dict(torch.load('./HW6_model/HW6_smooth_0.pkl'))
# model.eval()

In [35]:
test_q = pd.read_csv('./test_queries.csv')
test_query_list = test_q['query_id'].tolist()
test_query_data = [t.lower() for t in test_q['query_text']]
rerank_idx_list = [t.split() for t in test_q['bm25_top1000']]
rerank_score_list = [t.split() for t in test_q['bm25_top1000_scores']]
result_score_list = []
print(test_query_data[0])

international organized crime


In [36]:
print(rerank_idx_list[0][15])

FBIS3-21770


In [214]:
# test_question = []
# test_choice = []
# test_y = []
# for i in range(len(test_query_data)):
#     for j in range(len(rerank_idx_list[i])):
#         idx = doc_dict[rerank_idx_list[i][j]]
#         if type(doc_text[idx]) != float:
#             test_y.append(1)
#             test_question.append(test_query_data[i])
#             test_choice.append(clean_string(doc_text[idx]))

# # train_question = np.array(train_question)
# # train_choice = np.array(train_choice)
# # train_question = json.dumps(train_question)
# test_y = np.array(test_y)

In [37]:
result_score_list = np.zeros([80,1000]) ##分數
bert_score_list = np.zeros([80,1000])
for i in range(len(rerank_score_list)):
    for j in range(len(rerank_score_list[0])):
        result_score_list[i][j] = float(rerank_score_list[i][j])
after_add_list = np.zeros([80,1000])

In [26]:
print(result_score_list[0][15])

23.23352808


In [38]:
all_ans_list_1 = []
all_ans_list_2 = []
all_ans_list_3 = []
bert_ans_list = []
for i in range(80):
    print(i)
    bert_ans = []
    ans_list = []
#     ans_list2 = []
#     ans_list3 = []
    idx_list = rerank_idx_list[i]
    bert_score = bert_score_list[i].copy()
    score_list = result_score_list[i].copy()
    score_list2 = result_score_list[i].copy()
    score_list3 = result_score_list[i].copy()
#     denominator = score_list[0]/score_list[999]
#     score_list /= denominator
#     print(score_list)
    input_q = []
    input_c = []
    no_content_list = []
    for j in range(len(idx_list)):
        find = doc_dict[idx_list[j]] ## idx 
        if type(doc_text[find]) != float:
            text = clean_string(doc_text[find])
            input_q.append(test_query_data[i])
            input_c.append(text)
        else:
            no_content_list.append(j) ##空的
    batch_test_input_dict = tokenizer(input_q, input_c, 
                            padding=True, 
                            truncation=True, 
                            return_tensors="pt",
                            max_length = 512)
    TEST_BATCH_SIZE = 64
    batch_testset = TrainDataset(batch_test_input_dict,torch.zeros([len(idx_list)], dtype=torch.long)) ##trainset參數如init 給label沒差
    batch_testloader = DataLoader(batch_testset, batch_size = TEST_BATCH_SIZE, shuffle = False)  
    score = get_test_score(model,batch_testloader)
    no_content_count = 0
    for k in range(len(score)):
#         print(k)
#         print(score[k])
        if k not in no_content_list:
            bert_score[k+no_content_count] += score[k]
            score_list[k+no_content_count] +=  2 * score[k]
#             score_list2[k+no_content_count] +=  2 * score[k]
#             score_list3[k+no_content_count] +=  3 * score[k]
        else:
            no_content_count += 1
    after_add_list[i] = score_list
    bert_score_list[i] = bert_score
    sort_bert = sorted(range(1000),reverse = True,key = lambda k : bert_score[k])
    ans_idx_list = sorted(range(1000),reverse = True,key = lambda k : score_list[k])
#     ans_idx_list2 = sorted(range(1000),reverse = True,key = lambda k : score_list2[k])
#     ans_idx_list3 = sorted(range(1000),reverse = True,key = lambda k : score_list3[k])
    for j in range(1000):
        bert_ans.append(rerank_idx_list[i][sort_bert[j]])
        ans_list.append(rerank_idx_list[i][ans_idx_list[j]])
#         ans_list2.append(rerank_idx_list[i][ans_idx_list2[j]])
#         ans_list3.append(rerank_idx_list[i][ans_idx_list3[j]])
    bert_ans_list.append(bert_ans)
    all_ans_list_1.append(ans_list)
#     all_ans_list_2.append(ans_list2)
#     all_ans_list_3.append(ans_list3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [39]:
np.save('./HW6_model/bert_score_6',bert_score_list)

In [ ]:
test_score_list = np.zeros([25,1000]) ##分數
train_top_1000 = [t.split() for t in train_q['bm25_top1000']]
train_pos_list = [t.split() for t in train_q['pos_doc_ids']]
train_1000_score = [t.split() for t in train_q['bm25_top1000_scores']]
bert_test = np.zeros([25,1000])
for i in range(len(test_score_list)):
    for j in range(len(test_score_list[0])):
        test_score_list[i][j] = float(train_1000_score[i][j])
        
top_1000 = [t.split() for t in train_q['bm25_top1000']]
pos_list = [t.split() for t in train_q['pos_doc_ids']]
for i in range(25):
    print(i)
    idx_list = top_1000[i]
    bert_score = bert_test[i].copy()
    input_q = []
    input_c = []
    no_content_list = []
    for j in range(len(idx_list)):
        find = doc_dict[idx_list[j]] ## idx 
        if type(doc_text[find]) != float:
            text = clean_string(doc_text[find])
            input_q.append(test_query_data[i])
            input_c.append(text)
        else:
            no_content_list.append(j) ##空的
    batch_test_input_dict = tokenizer(input_q, input_c, 
                            padding=True, 
                            truncation=True, 
                            return_tensors="pt",
                            max_length = 512)
    TEST_BATCH_SIZE = 64
    batch_testset = TrainDataset(batch_test_input_dict,torch.zeros([len(idx_list)], dtype=torch.long)) ##trainset參數如init 給label沒差
    batch_testloader = DataLoader(batch_testset, batch_size = TEST_BATCH_SIZE, shuffle = False)  
    score = get_test_score(model,batch_testloader)
    no_content_count = 0
    for k in range(len(score)):
        if k not in no_content_list:
            bert_score[k+no_content_count] += score[k]
        else:
            no_content_count += 1
    bert_test[i] = bert_score ##塞回去

In [306]:
test_score_list = np.zeros([25,1000]) ##分數
train_top_1000 = [t.split() for t in train_q['bm25_top1000']]
train_pos_list = [t.split() for t in train_q['pos_doc_ids']]
train_1000_score = [t.split() for t in train_q['bm25_top1000_scores']]
for i in range(len(test_score_list)):
    for j in range(len(test_score_list[0])):
        test_score_list[i][j] = float(train_1000_score[i][j])

In [307]:
a = -2 ## alpha
n = 1000
delta = 0.01
top = [t.split() for t in train_q['bm25_top1000']]
ans = [t.split() for t in train_q['pos_doc_ids']]
for num in range(n): #每次加delta:
    pred_list = []
#     all_score = test_score_list + a * bert_test
    ans_list = ans[:20]
    for i in range(20):
        bert_ans = []
        all_score = test_score_list[i] + a * bert_test[i]
        score = sorted(range(1000),reverse = True,key = lambda k : all_score[k])
        for j in range(1000):
            bert_ans.append(top[i][score[j]])
        pred_list.append(bert_ans)
#     print(pred_list)
    result = MAP(pred_list) ##要傳25x1000
    a += delta
    print('a:',a,'  MAP:',result)

a: -1.99   MAP: 0.2541
a: -1.98   MAP: 0.2543
a: -1.97   MAP: 0.2543
a: -1.96   MAP: 0.2543
a: -1.95   MAP: 0.2544
a: -1.94   MAP: 0.2545
a: -1.93   MAP: 0.2545
a: -1.92   MAP: 0.2545
a: -1.91   MAP: 0.2544
a: -1.9   MAP: 0.2544
a: -1.89   MAP: 0.2545
a: -1.88   MAP: 0.2545
a: -1.8699999999999999   MAP: 0.2546
a: -1.8599999999999999   MAP: 0.2546
a: -1.8499999999999999   MAP: 0.2547
a: -1.8399999999999999   MAP: 0.2548
a: -1.8299999999999998   MAP: 0.2548
a: -1.8199999999999998   MAP: 0.2563
a: -1.8099999999999998   MAP: 0.2563
a: -1.7999999999999998   MAP: 0.2556
a: -1.7899999999999998   MAP: 0.2557
a: -1.7799999999999998   MAP: 0.2557
a: -1.7699999999999998   MAP: 0.2558
a: -1.7599999999999998   MAP: 0.2552
a: -1.7499999999999998   MAP: 0.2553
a: -1.7399999999999998   MAP: 0.2553
a: -1.7299999999999998   MAP: 0.2553
a: -1.7199999999999998   MAP: 0.2554
a: -1.7099999999999997   MAP: 0.2553
a: -1.6999999999999997   MAP: 0.2554
a: -1.6899999999999997   MAP: 0.2554
a: -1.6799999999999997

a: 0.610000000000002   MAP: 0.2599
a: 0.620000000000002   MAP: 0.2599
a: 0.630000000000002   MAP: 0.2599
a: 0.640000000000002   MAP: 0.2599
a: 0.650000000000002   MAP: 0.2599
a: 0.660000000000002   MAP: 0.2599
a: 0.670000000000002   MAP: 0.2594
a: 0.680000000000002   MAP: 0.259
a: 0.6900000000000021   MAP: 0.259
a: 0.7000000000000021   MAP: 0.259
a: 0.7100000000000021   MAP: 0.259
a: 0.7200000000000021   MAP: 0.259
a: 0.7300000000000021   MAP: 0.2583
a: 0.7400000000000021   MAP: 0.2582
a: 0.7500000000000021   MAP: 0.2582
a: 0.7600000000000021   MAP: 0.2581
a: 0.7700000000000021   MAP: 0.2582
a: 0.7800000000000021   MAP: 0.2581
a: 0.7900000000000021   MAP: 0.2579
a: 0.8000000000000022   MAP: 0.2579
a: 0.8100000000000022   MAP: 0.2578
a: 0.8200000000000022   MAP: 0.2577
a: 0.8300000000000022   MAP: 0.2577
a: 0.8400000000000022   MAP: 0.2577
a: 0.8500000000000022   MAP: 0.2577
a: 0.8600000000000022   MAP: 0.2577
a: 0.8700000000000022   MAP: 0.2576
a: 0.8800000000000022   MAP: 0.2576
a: 0.

a: 3.789999999999965   MAP: 0.2009
a: 3.7999999999999647   MAP: 0.2007
a: 3.8099999999999645   MAP: 0.2005
a: 3.8199999999999643   MAP: 0.2004
a: 3.829999999999964   MAP: 0.2
a: 3.839999999999964   MAP: 0.1997
a: 3.8499999999999637   MAP: 0.1996
a: 3.8599999999999635   MAP: 0.1993
a: 3.8699999999999632   MAP: 0.1991
a: 3.879999999999963   MAP: 0.199
a: 3.889999999999963   MAP: 0.1987
a: 3.8999999999999626   MAP: 0.1984
a: 3.9099999999999624   MAP: 0.1983
a: 3.919999999999962   MAP: 0.1982
a: 3.929999999999962   MAP: 0.198
a: 3.9399999999999618   MAP: 0.1978
a: 3.9499999999999615   MAP: 0.1977
a: 3.9599999999999613   MAP: 0.1975
a: 3.969999999999961   MAP: 0.1974
a: 3.979999999999961   MAP: 0.1971
a: 3.9899999999999607   MAP: 0.1969
a: 3.9999999999999605   MAP: 0.1966
a: 4.009999999999961   MAP: 0.1964
a: 4.0199999999999605   MAP: 0.1962
a: 4.02999999999996   MAP: 0.196
a: 4.03999999999996   MAP: 0.1959
a: 4.04999999999996   MAP: 0.1956
a: 4.05999999999996   MAP: 0.1955
a: 4.06999999999

a: 6.249999999999913   MAP: 0.1461
a: 6.259999999999913   MAP: 0.1459
a: 6.2699999999999125   MAP: 0.1458
a: 6.279999999999912   MAP: 0.1456
a: 6.289999999999912   MAP: 0.1455
a: 6.299999999999912   MAP: 0.1454
a: 6.309999999999912   MAP: 0.144
a: 6.3199999999999115   MAP: 0.1438
a: 6.329999999999911   MAP: 0.1437
a: 6.339999999999911   MAP: 0.1435
a: 6.349999999999911   MAP: 0.1435
a: 6.359999999999911   MAP: 0.1434
a: 6.36999999999991   MAP: 0.1432
a: 6.37999999999991   MAP: 0.1431
a: 6.38999999999991   MAP: 0.1429
a: 6.39999999999991   MAP: 0.1428
a: 6.4099999999999095   MAP: 0.1427
a: 6.419999999999909   MAP: 0.1425
a: 6.429999999999909   MAP: 0.1424
a: 6.439999999999909   MAP: 0.1423
a: 6.449999999999909   MAP: 0.1422
a: 6.4599999999999085   MAP: 0.1421
a: 6.469999999999908   MAP: 0.142
a: 6.479999999999908   MAP: 0.1418
a: 6.489999999999908   MAP: 0.1417
a: 6.499999999999908   MAP: 0.1416
a: 6.509999999999907   MAP: 0.1415
a: 6.519999999999907   MAP: 0.1415
a: 6.529999999999907  

In [285]:
with open('./hw6_result_BM25_Bert_a_2_84.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(all_ans_list_1[0])):
            f.write(all_ans_list_1[i][j])
            f.write(' ')
        f.write('\n')
    f.close()

In [183]:
with open('./hw6_result_BM25_Bert_a_2_all_epoch3.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(all_ans_list_2[0])):
            f.write(all_ans_list_2[i][j])
            f.write(' ')
        f.write('\n')
    f.close()

In [184]:
with open('./hw6_result_BM25_Bert_a_3_all_epoch3.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(all_ans_list_3[0])):
            f.write(all_ans_list_3[i][j])
            f.write(' ')
        f.write('\n')
    f.close()

In [219]:
with open('./hw6_only_Bert.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(bert_ans_list[0])):
            f.write(bert_ans_list[i][j])
            f.write(' ')
        f.write('\n')
    f.close()

In [48]:
score_1 = np.load('./HW6_model/84_bert_score.npy')
score_2 = np.load('./HW6_model/85_bert_score.npy')
score_3 = np.load('./HW6_model/new_bert_score.npy')
score_4 = np.load('./HW6_model/bert_score_3.npy')
score_5 = np.load('./HW6_model/bert_score_4.npy')
score_6 = np.load('./HW6_model/91_bert_score.npy')
score_7 = np.load('./HW6_model/bert_score_6.npy')

In [40]:
print(bert_score_list[0].min())
print(bert_score_list[0].max())

-3.2034130096435547
2.0394632816314697


In [49]:
# print(bert_score_list[0][:10])
# print(bert_score_list[0].min())
# print(bert_score_list[0].max())
print(score_1[0][:10])
print(score_1[0].min())
print(score_1[0].max())
print(score_2[0][:10])
print(score_2[0].min())
print(score_2[0].max())
print(score_3[0][:10])
print(score_3[0].min())
print(score_3[0].max())
print(score_4[0][:10])
print(score_4[0].min())
print(score_4[0].max())
print(score_5[0][:10])
print(score_5[0].min())
print(score_5[0].max())
print(score_6[0][:10])
print(score_6[0].min())
print(score_6[0].max())
print(score_7[0][:10])
print(score_7[0].min())
print(score_7[0].max())

[ 0.19857574  0.2864981   0.30835772  0.91545808  0.44452697  0.56130213
 -0.76218355  0.93669212  0.93564975 -0.08939899]
-2.9418857097625732
0.977552592754364
[-0.01611805  1.01507843  0.98575181  1.76120901  0.82554829  0.46683502
 -0.00744006  1.16104364  1.16643512 -0.38859686]
-4.135344505310059
2.5401198863983154
[ 0.32602665  0.19839273  0.20481455  0.15634261  0.12310709 -0.04645513
 -0.44732541  0.44058132  0.44106436  0.29140231]
-3.106045961380005
0.808659553527832
[ 0.46305868  1.10627103  1.37598395  2.11274195  0.34400618  0.68545663
  1.26805615  1.45703733  1.4523679  -0.27640003]
-3.957559823989868
3.1810543537139893
[0.20581445 0.57972723 0.58888334 0.65538132 0.24105221 0.56531078
 0.170351   0.93586588 0.93555534 0.54967535]
-2.6166515350341797
0.9358658790588379
[0.77312845 1.26064122 1.25226676 1.91743433 0.48077983 0.66892183
 0.63924706 1.62464654 1.62473249 0.41933694]
-3.815316677093506
1.9174343347549438
[1.39012039 1.87458003 1.90994799 1.96015906 1.3477598

In [50]:
score_2/=2.5
score_4/=3
score_6/=1.9
score_7/=2

In [51]:
mix_score_list = np.zeros([80,1000])

In [52]:
mix_score_list += score_1  + score_2 +score_3+score_4+score_5+score_6+score_7

In [53]:
mix_score_list /= 7

In [57]:
print(mix_score_list[0])
print(result_score_list[0])
all_ans = []
for i in range(80):
    ans = []
    score = result_score_list[i] + (8 * mix_score_list[i])
    idx = sorted(range(1000),reverse = True,key = lambda k : score[k])
    for j in range(1000):
        ans.append(rerank_idx_list[i][idx[j]])
    all_ans.append(ans)

[ 0.28289892  0.49145598  0.50986849  0.73216692  0.31149946  0.3520447
 -0.01214493  0.72351912  0.72362196  0.2120355   0.17472876  0.17615887
  0.22143613  0.00382071  0.63152177  0.00795055 -0.13272919  0.06878403
  0.06633442 -0.58717008  0.29697921  0.40608862  0.05354757  0.05366373
 -0.07297313  0.3504502   0.10580328 -1.31759286  0.13083852 -0.09528823
 -0.83777497 -0.83777497  0.37607381  0.40727685 -1.06522253 -1.04820595
  0.22160451 -1.48810775  0.62143796 -0.8103536  -1.5762138  -1.29638347
 -1.08198174  0.49371712 -1.51573341 -0.44404663 -0.96617319 -1.71967043
 -1.85620761  0.07164099 -1.54416393 -0.50167333 -0.44353503 -1.25204521
  0.25100582 -0.26132128 -1.36419886 -1.2461449   0.14055153 -0.62048992
 -1.69575786 -0.81687692  0.04527056 -1.62359103 -1.46975984  0.21796911
 -0.14963541 -0.58200839 -0.53468115 -0.14709225 -1.00771958  0.39037599
 -0.34777678 -0.40963636 -1.04727652 -1.65254352 -0.78371078 -0.62158299
 -0.65377924  0.20909841 -1.35314226 -1.35574977 -0.

In [58]:
with open('./hw6_new_best_91.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(all_ans[0])):
            f.write(all_ans[i][j])
            f.write(' ')
        f.write('\n')
    f.close()